In [15]:
import pandas as pd
pd.set_option('display.max_colwidth', 50)

data = pd.read_pickle('pickle/articles.pkl')
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 85 entries, 0 to 85
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        85 non-null     object
 1   description  85 non-null     object
 2   date         85 non-null     object
dtypes: object(3)
memory usage: 2.7+ KB


In [16]:
import flair
from flair.data import Sentence
sentiment_model = flair.models.TextClassifier.load('en-sentiment')

2020-12-17 12:52:39,851 loading file /Users/alexandre/.flair/models/sentiment-en-mix-distillbert_3.1.pt


In [17]:
def sentiment_per_cat(df, cat):
    sentiment = []
    confidence = []
    for c in data[cat]:
        sent = Sentence(c)
        sentiment_model.predict(sent)
        sentiment.append(sent.labels[0].value)
        confidence.append(sent.labels[0].score)
    df["sentiment_" + cat] = sentiment
    df["confidence_" + cat] = confidence
    
    return df

In [18]:
data

,title,description,date
0,columngetting mighty crowded searching for ma...,the biggest fear in markets is often of market...,2020-12-09
1,column getting mighty crowded searching for m...,column getting mighty crowded searching for m...,2020-12-09
2,bitcoin ethereum ripples xrp litecoin and chai...,bitcoin has fallen sharply over the last 24 ho...,2020-12-09
3,the grumpy designer looks ahead to 2021,consider this the definitive tongueincheek loo...,2020-12-09
4,4 gamechanging stocks to invest 1000 in right now,these innovative companies can make investors ...,2020-12-09
...,...,...,...
81,bitcoin tops 20k for the first time cnet,the cryptocurrency has hit a record high,2020-12-16
82,jeff reevess strength in numbers i admit it — ...,jeff reeves says he was wrong about covid19s i...,2020-12-16
83,how i blew my bitcoin on sushi,in 2013 this reporter spent 10 bitcoin worth 1...,2020-12-16
84,bitcoin at alltime high of 21300 as institutio...,the price of bitcoin has been steadily on the ...,2020-12-16


In [19]:
# Analyse des titres
sentiment_per_cat(data, "title")
# Analyse des descriptions
sentiment_per_cat(data, "description")

,title,description,date,sentiment_title,confidence_title,sentiment_description,confidence_description
0,columngetting mighty crowded searching for ma...,the biggest fear in markets is often of market...,2020-12-09,NEGATIVE,0.987236,NEGATIVE,0.959164
1,column getting mighty crowded searching for m...,column getting mighty crowded searching for m...,2020-12-09,NEGATIVE,0.999172,NEGATIVE,0.999172
2,bitcoin ethereum ripples xrp litecoin and chai...,bitcoin has fallen sharply over the last 24 ho...,2020-12-09,POSITIVE,0.847576,NEGATIVE,0.999955
3,the grumpy designer looks ahead to 2021,consider this the definitive tongueincheek loo...,2020-12-09,NEGATIVE,0.987747,POSITIVE,0.996804
4,4 gamechanging stocks to invest 1000 in right now,these innovative companies can make investors ...,2020-12-09,POSITIVE,0.951164,POSITIVE,0.998714
...,...,...,...,...,...,...,...
81,bitcoin tops 20k for the first time cnet,the cryptocurrency has hit a record high,2020-12-16,POSITIVE,0.921074,POSITIVE,0.995639
82,jeff reevess strength in numbers i admit it — ...,jeff reeves says he was wrong about covid19s i...,2020-12-16,NEGATIVE,0.996728,NEGATIVE,0.998286
83,how i blew my bitcoin on sushi,in 2013 this reporter spent 10 bitcoin worth 1...,2020-12-16,POSITIVE,0.533619,NEGATIVE,0.664276
84,bitcoin at alltime high of 21300 as institutio...,the price of bitcoin has been steadily on the ...,2020-12-16,NEGATIVE,0.996455,NEGATIVE,0.990359


In [20]:
# Remove low confidence score
masque1 = data.confidence_title < 0.9
data = data.drop(data[masque1].index, axis=0)
masque2 = data.confidence_description < 0.8
data = data.drop(data[masque2].index, axis=0)

# Remove article with oposite sentiment from title and description
masque3 = data.sentiment_title != data.sentiment_description
data = data.drop(data[masque3].index, axis=0)
data = data.reset_index(drop = True)

In [21]:
data

,title,description,date,sentiment_title,confidence_title,sentiment_description,confidence_description
0,columngetting mighty crowded searching for ma...,the biggest fear in markets is often of market...,2020-12-09,NEGATIVE,0.987236,NEGATIVE,0.959164
1,column getting mighty crowded searching for m...,column getting mighty crowded searching for m...,2020-12-09,NEGATIVE,0.999172,NEGATIVE,0.999172
2,4 gamechanging stocks to invest 1000 in right now,these innovative companies can make investors ...,2020-12-09,POSITIVE,0.951164,POSITIVE,0.998714
3,global stocks rise on higher white house stimu...,summary list placementulliglobal stocks rose o...,2020-12-09,NEGATIVE,0.902753,NEGATIVE,0.999673
4,with investors expecting a latin american cryp...,six years after the launch of the mexicobased ...,2020-12-09,NEGATIVE,0.992876,NEGATIVE,0.990348
5,coinbases backstory and future with kings of c...,i used coinbase as a vehicle because a lot of ...,2020-12-09,POSITIVE,0.993253,POSITIVE,0.998374
6,jpmorgan ceo jamie dimon called treasurys a po...,summary list placementullijpmorgans jamie dimo...,2020-12-09,NEGATIVE,0.999912,NEGATIVE,0.999934
7,bitcoins intrinsic value will rise in the comi...,summary list placementullijpmorgan strategists...,2020-12-09,NEGATIVE,0.996448,NEGATIVE,0.999980
8,why cryptocurrency stocks crashed today,cryptocurrency stocks and funds such as graysc...,2020-12-09,NEGATIVE,0.999369,NEGATIVE,0.999872
9,forexpound drops as brexit impasse unsettled a...,the british pound lost about 05 on thursday af...,2020-12-09,NEGATIVE,0.984834,NEGATIVE,0.998717


In [22]:
data.to_pickle("pickle/articles_sentiment_analysis.pkl")